# Vergleich ECM und Bucket-Modell


In [1]:
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from scipy.optimize import fsolve


In [2]:
pd.options.plotting.backend = "plotly"
template= "plotly_white"

In [3]:
def minmax(lb, v, ub):
    return min(max(v, lb), ub)

# Storage Models

## Bucket Model

In [4]:
# params
cell_nominal_energy = 3.0 * 3.2 # Ah * V -> Wh

In [5]:
def bucket_update(soc, power_target, capacity, max_power, dt):
    power = minmax(-max_power, power_target, max_power)
    soc_new = minmax(0, soc - power * dt / capacity, 1)
    power = (soc - soc_new) / dt * capacity
    fullfilment = power_target / power
    fullfilment = round(fullfilment, 4)

    # TODO: efficiency
    return dict(
        soc_new=soc_new,
        power=power,
        # power_target=power_target,
        fullfilment=fullfilment
    )

In [6]:
def simulate_bucket(power_profile, capacity, max_power, initial_soc, dt):
    df = power_profile.copy()
    df["power"] = 0.0
    df["soc"] = 0.0
    # TODO: add fullfilment factor

    soc = initial_soc

    for time, power_target in power_profile.items():
        data = bucket_update(soc, power_target, capacity, max_power, dt)
        df = pd.concat([df, pd.DataFrame(index=[time], data=[data])])
        soc = data["soc"]


    return df

## Equivalent-Circuit Model

In [7]:
# params
cell_nominal_capacity = 3.0  # Ah
cell_nominal_voltage = 3.2  # V
cell_icmax = cell_nominal_capacity * 1.0  # 1.0 C
cell_idmax = cell_nominal_capacity * 6.6  # 6.6 C
r = 0.05 # internal resistance in Ohm

# TODO: 
# - scale cell to storage system (parallel and serial connections to fullfill capacity and voltage)
# - cell parameters as argumetns to update function? or global variables?

In [8]:
def circuit(capacity, voltage):
    parallel = voltage / cell_nominal_voltage
    serial = capacity / voltage / cell_nominal_capacity
    return parallel, serial

In [9]:
parallel, serial = circuit(100, 300)

### Open-circuit-voltage

In [10]:
def open_circuit_voltage(soc):
    a1 = -116.2064
    a2 = -22.4512
    a3 = 358.9072
    a4 = 499.9994
    b1 = -0.1572
    b2 = -0.0944
    k0 = 2.0020
    k1 = -3.3160
    k2 = 4.9996
    k3 = -0.4574
    k4 = -1.3646
    k5 = 0.1251

    return (
        k0
        + k1 / (1 + np.exp(a1 * (soc - b1)))
        + k2 / (1 + np.exp(a2 * (soc - b2)))
        + k3 / (1 + np.exp(a3 * (soc - 1)))
        + k4 / (1 + np.exp(a4 * soc))
        + k5 * soc
    )

In [11]:
soc_range = np.linspace(start=0.0, stop=1.0, num=1000)
ocv_curve = open_circuit_voltage(soc_range)
px.line(x=soc_range, y=ocv_curve, template=template, labels={"x": "SOC in p.u.", "y": "OCV in V"})

### Model

In [12]:
def ecm_update(soc, power_target, capacity, max_current, dt, r):
        p = power_target
        ocv = open_circuit_voltage(soc)

        i = p / ocv  # initial guess
        i = fsolve(lambda i: p - (ocv - i * r) * i, x0=i) # p - (voc - i * r) * i = 0
        i = i [0] # unpack array
        i = minmax(-max_current, i, max_current)

        soc_new = minmax(0, soc - i * dt / capacity , 1)
        
        v = ocv - i * r
        p = v * i
        fullfilment = power_target / p
        fullfilment = round(fullfilment, 4)
        # print(f"{i=:.4f} {ocv=:.4f} {v=:.4f} {p=:.4f} {soc_new=:.4f}")
        
        return {
                "power": p,
                "current": i,
                "voltage": v,
                "ocv": ocv,
                "soc": soc_new,
                "fullfilment": fullfilment
        }

In [13]:
def simulate_ecm(power_profile, capacity, max_current, initial_soc, dt):
    df = pd.DataFrame()
    soc = initial_soc
    r = 0.05

    for time, power_target in power_profile.items():
        data = ecm_update(soc, power_target, capacity, max_current, dt, r)
        df = pd.concat([df, pd.DataFrame(index=[time], data=[data])])
        soc = data["soc"]

    return df

### Energy delivered by *"C-rate"*

In [14]:
def constant_power_discharge(c_rate, capacity=3.0, dt=1/60):
    soc = 1
    power_target = capacity * c_rate
    max_current = capacity * 6.0
    
    df = pd.DataFrame()
    t = 0

    while soc > 0: # better a precision eps > 0 ?
        data = ecm_update(soc=soc, power_target=power_target, dt=dt, capacity=capacity, max_current=max_current, r=r)
        soc = data["soc"]
        t += dt
        df = pd.concat([df, pd.DataFrame(data=[data], index=[t])])

    return df

In [15]:
c_rates = [1/60, 1/8, 1/4, 1/2, 1, 2]
series = {c_rate: constant_power_discharge(c_rate) for c_rate in c_rates}

energy_1c = series[1]["power"].sum() * 1/60
print("Total energy discharged (normalized to 1C):")
for c, s in series.items():
    print(f"{c:.2f}C: {s['power'].sum() * (1/60) / energy_1c:.4f}")

Total energy discharged (normalized to 1C):
0.02C: 1.0095
0.12C: 1.0084
0.25C: 1.0064
0.50C: 1.0052
1.00C: 1.0000
2.00C: 0.9897


In [16]:
def plot_curves(series, **kwargs):
    fig = go.Figure()
    fig.update_layout(xaxis_title="", yaxis_title="Voltage [V]", **kwargs)
    fig.add_trace(go.Scatter(x=np.arange(8760), y=np.zeros(8760), line={"color": "grey", "dash": "dash"}, opacity=0.7, showlegend=False, name=0))
    for crate, df in series.items():
        voltage=df.voltage
        #time_hours = np.arange(aggregated_residual.size)
        fig.add_trace(go.Scatter(y=voltage, name= crate))
    return fig

In [17]:
plot_curves(series, title="C-rate dependent discharge", width=700, height=600, template=template)
#df1.plot()